# data understanding of engineered features
This notebook takes care of the data understanding done on engineered features 

In [112]:
from os import path
import  pandas as pd
import seaborn 
import matplotlib.pyplot as plt
import numpy as np
ENGINEERED_RACES_PATH=path.join("..","dataset","engineered_races.csv")

The analysis is divided in three parts/comparisons:
- new vs old
- old vs old
- new vs new

In [ ]:
races_df=pd.read_csv(ENGINEERED_RACES_PATH)
races_df['date']=pd.to_datetime(races_df['date'])


oldf_df=races_df[[        
        'points', 
        'length', 
        'climb_total', 
        'profile', 
        'startlist_quality',
        'position', 
        'cyclist_age', 
        'is_tarmac',
        'delta'
        ]]

newf_df=races_df[[
        'climbing_efficiency', 
        'competitive_age',
        'difficulty', 
        'convenience_score',
        'difficulty_score', 
        'performance_index',
        'gain_ratio'
        ]]
on_corr_data=oldf_df.corrwith(newf_df,axis=1)
old_corr_data=oldf_df.corr()
new_corr_data=newf_df.corr()
print(corr_data)
plt.figure(figsize=(12,10))
seaborn.heatmap(
    corr_data,
    annot=True, cmap='magma', vmin=-1, vmax=1, linewidths=1,fmt='.2f')


0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
589860   NaN
589861   NaN
589862   NaN
589863   NaN
589864   NaN
Length: 589865, dtype: float64


IndexError: Inconsistent shape between the condition and the input (got (589865, 1) and (589865,))

<Figure size 1200x1000 with 0 Axes>

From a first raw plot we can see two unexpected results there a really negative correlation between performance_index an position and also between is_tarmac and points, the first one is to be expected of course the performance gets worse as the position is further back in the race, however is_tarmac and points is very strange to explain.

the correlation between is_tarmac and points might be explained by the ffact that we binarized the attrtibute, since the other is_X types are never true for us, and it might the result of an approximation without any semantical meaning,if a meaning is there might be that tarmac makes the difficulty increase but further investigations are needed to find any useful deduction to make.

The difficulty score is of course  related to length , climb_total and profile since it is calculated using them however we see a correlation using delta which obvious given the correlation with how much time it takes to travel a difficult course whether it is due to steepness, high quotas or length.

Please note that the difficulty score is only related to races and not the single player capabilities, wee see a really low correlation with the difficulty which is also dependent on the single racer characteristics this fact suggests that difficulty is highly subjective to each player given its capabilities and physical attributes at the time the  race happens.

We get that lengths and points are highly correlated which means that a longer race has more prestige and thus it awards more points, we also have a positive correlation 